### CCE Data - Preliminary data cleanup and MDS plots  
Authors: Abzer Kelminal (abzer.shah@uni-tuebingen.de) <br>
Edited by:    <br>
Input file format: .csv files directly from github <br> 
Outputs: .csv files and .svg files(plots) <br>
Dependencies: ggplot2, dplyr, ecodist, RcolorBrewer, svglite

In [ ]:
# Install the needed packages:

#install.packages("dplyr")
#install.packages("ggplot2")
#install.packages("ecodist")  #For PcoA using Bray-Curtis distance
#install.packages("RColorBrewer") # to use Colorblindness-friendly colors
#install.packages("svglite") # for saving ggplots as svg

In [ ]:
# calling the libraries:
library(dplyr)
library(ggplot2)
library(ecodist) 
library(RColorBrewer)
library(svglite)

In [ ]:
# Directly calling the files from the github account:
ft_url <- "https://raw.githubusercontent.com/Functional-Metabolomics-Lab/CCE_Data-Analysis/main/raw%20files/CCE1706_MZmine3_GNPS_1_quant.csv"
md_url <- "https://raw.githubusercontent.com/Functional-Metabolomics-Lab/CCE_Data-Analysis/main/raw%20files/metadata_CCE.csv"

ft <- read.csv(ft_url, header = T, check.names = F)
md <- read.csv(md_url, header = T, check.names = F)

In [ ]:
head(ft)

In [ ]:
head(md)

In [ ]:
# Arranging the metadata table in the right format for further analysis:

md <- md[,colSums(is.na(md))<nrow(md)] #removing NA columns in md, if any present
rownames(md) <- md$filename 
md <- md[,-1]
rownames(md) <- gsub('Blank_','',rownames(md)) # Removing "Blank" in the 1st two row names of md

In [ ]:
head(md)

In [ ]:
# Arranging the feature table in the right format for further analysis:

new_ft <- ft[,grep('mzXML',colnames(ft))] # only picking mzXML files
new_ft <- new_ft[,-grep('_STKL',colnames(new_ft))] # excluding the STKL columns
colnames(new_ft) <- gsub('_MSMS.mzXML Peak area','.mzxml',colnames(new_ft))  #substituting the file extension in the colnames of new_ft with mzxml
rownames(new_ft) <- paste(ft$'row ID',round(ft$'row m/z',digits = 3),round(ft$'row retention time',digits = 3), sep = '_') #Taking row ID, m/z value and RT of ft as the rownames of new_ft

new_ft <- new_ft[,order(colnames(new_ft))] #ordering the columns by names
new_ft <- new_ft[,-1] # Removing the column "Brandon_Deep_DOM"

In [ ]:
head(new_ft)

In [ ]:
ft_final <- new_ft[,which(colnames(new_ft)%in%rownames(md))] #picking only the columns in new_ft that are comparable to rownames of md
ft_final  <- new_ft[,match(rownames(md),colnames(new_ft))] #returns the matched position of rownames of md to that of colnames of new_ft
identical(colnames(ft_final),rownames(md)) #checking if the colnames of ft_final and rownames of md matched. Should return TRUE

In [ ]:
dim(ft_final)
dim(md)

The dimensions show that nrow(ft_final)=ncol(md), thus eligible for matrix multiplication. Therefore, we can apply several calculations on these tables such as PCA etc.

### Creating a function named FrequencyPlot:  
The below function takes in the two input datatables: gapfilled and non-gapfilled, calculates the frequency distribution of the data in the order of 10 and produces a grouped barplot showing the distribution as output. The frequency plot shows where the features are present in higher number.

In [ ]:
options(repr.plot.width=5, repr.plot.height=3) #'global' settings for plot size in the output cell

FrequencyPlot <- function(x1,x2){
    
    #creating bins from -1 to 10^10 using sequence function seq()
    bins <- c(-1,0,(1 * 10^(seq(0,10,1)))) 
    
    #cut function cuts the give table into its appropriate bins
    scores_x1 <- cut(as.matrix(x1),bins,labels = c('0','1','10','1E2','1E3','1E4','1E5','1E6','1E7','1E8','1E9','1E10')) 
    
    #transform function convert the tables into a column format: easy for visualization 
    Table_x1<-transform(table(scores_x1)) #contains 2 columns: "scores_x1", "Freq"
    
    #Repeating the same steps for x2
    scores_x2 <- cut(as.matrix(x2),bins,labels = c('0','1','10','1E2','1E3','1E4','1E5','1E6','1E7','1E8','1E9','1E10'))
    Table_x2<-transform(table(scores_x2))
  
    #Getting the names of x1 and x2
    arg1 <- deparse(substitute(x1))
    arg2 <-deparse(substitute(x2))
    
    #Creating a data frame for plotting
    data_plot <- as.data.frame(c(Table_x1$Freq,Table_x2$Freq)) #Concatenating the frequency info of both tables rowwise
    colnames(data_plot) <- "Freq" #naming the 1st column as 'Freq'
    data_plot$Condition <- c(rep(arg1,12),rep(arg2,12)) #adding a 2nd column 'Condition', which just repeats the name of x1 and x2 accordingly
    data_plot$Range_bins <- rep(Table_x1$scores_x1,2) #Adding 3rd column 'Range Bins'
    data_plot$Log_Freq <- log(data_plot$Freq+1) #Log scaling the frequency values
    
    ## GGPLOT2
    BarPlot <- ggplot(data_plot, aes(Range_bins, Log_Freq, fill = Condition)) + 
    geom_bar(stat="identity", position = "dodge", width=0.4) + 
    scale_fill_brewer(palette = "Set1") +
    ggtitle(label="Frequency plot") +
    xlab("Range") + ylab("(Log)Frequency") + labs(fill = "Data Type") + 
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) +   # setting the angle for the x label
    theme(axis.text.y = element_text(angle = 45, vjust = 0.5, hjust=1)) +   # setting the angle for the y label
    theme(plot.title = element_text(hjust = 0.5)) # centering the plot title
  
    print(BarPlot)
}  

## Blank Removal:

(Note: In LC-MS/MS, we use solvents also called as Blanks which are usually injected time-to-time to prevent carryover of the sample)

The blanks we are referring to here, is the control blanks in the experiment and not the LCMSMS blanks.
- The control blanks here is the sample without treatment. 
- Samples are biological replicates with some treatment 

For the next step, Blank removal, we need to split the data as control and samples.

In general, having multiple control blanks helps us to compare any variation in the data. Comparing control to the sample helps us to identify the background features that contribute to any technical variation.  
A common filtering method is to use a cutoff to remove features that are not present sufficient enough in biological samples.

1. We find an average for all the feature intensities in your control set and sample set.
Therefore, for n no.of features in a control or sample set, we get n no.of averaged features.
2. Next, we get a ratio of this average_control vs average_sample. This ratio Control/sample tells us how much of that particular feature of a sample gets its contribution from control. If it is more than 30% (or Cutoff as 0.3), we consider the feature as noise.
3. The resultant information (if ratio > Cutoff or not) is stored in a bin
4. We count the no.of features in the bin that satisfies the condition ratio > cutoff, and consider those features as 'noise or background features'.
5. We also try to visualize the frequency distribution in our data Ctrl and samples

For a dataset containing several batches, the filtering steps are performed batch-wise.

In [ ]:
#Splitting the samples: 
Ctrl <-  ft_final[,1:2]
Samples <- ft_final[,-1:-2]

For the code below, <font color='red'> I selected 'Y' and gave a cutoff value: 0.3 </font>

In [ ]:
input_data <- ft_final

if(readline('Do you want to perform Blank Removal Process - Y/N:')=='Y'){
  Cutoff <- as.numeric(readline('Enter Cutoff value between 0.1 & 1:')) # (i.e. 10% - 100%). Ideal cutoff range: 0.1-0.3
  #When cutoff is low, more noise (or background) detected; With higher cutoff, less background detected, thus more features observed
  
  Avg_ctrl <- rowMeans(Ctrl, na.rm= FALSE, dims = 1) # set na.rm = FALSE to check if there are NA values. Because when set as TRUE, NA values are changed to 0
  Avg_samples <- rowMeans(Samples, na.rm= FALSE, dims = 1)
  Ratio_Ctrl_Sample <- (Avg_ctrl+1)/(Avg_samples+1)
  Bg_bin <- ifelse(Ratio_Ctrl_Sample > Cutoff, 1, 0 )  # checks if the Ratio is greater than Cutoff, if so put 1, else 0 in Bg_bin
  
  # to check if there are any NA values present. It is not good to have NA values in the 4 variables as it will affect the final dataset to be created
  temp_NA_Count <-cbind(Avg_ctrl ,Avg_samples,Ratio_Ctrl_Sample,Bg_bin)
  
  print('No of NA values in the following columns:')
  print(colSums(is.na(temp_NA_Count)))
  
  Bg_Features <- sum(Bg_bin ==1,na.rm = TRUE) # Calculates the number of background features present
  No_of_Features <- nrow(input_data) - Bg_Features
  
  print(paste("No.of Background or noise features:",Bg_Features))
  print(paste("No.of features after excluding noise:",No_of_Features)) 
  
  input_data1 <- cbind(as.matrix(input_data),Bg_bin)    
  plot_CtrlSample <- FrequencyPlot(Samples,Ctrl)
}

## Imputation:
For several reasons, real world datasets might have some missing values in it, in the form of **NA, NANs or 0s**. Eventhough the gapfillig step of MZmine fills the missing values, we still end up with some missing values or Os in our feature table. This could be problematic for statistical analysis. In order to have a better dataset, we cannot simply discard those rows or columns with missing values as we will lose a chunk of our valuable data. Instead we can try imputing those missing values. Imputation involves replacing the missing values in the data with a meaningful, reasonable guess. There are several methods, such as:

- Mean imputation (replacing the missing values in a column with the mean or average of the column)
- Replacing it with the most frequent value
- Several other machine learning imputation methods such as k-nearest neighbors algorithm(k-NN), Hidden Markov Model(HMM)

The method that we use is: Replacing the zeros from the gapfilled quant table with the Cutoff_LOD value.

### Continuing from previous steps in Blank Removal :  
6. **From the plot, we decide on a cutoff_LOD value (LOD-Limit of Detection).** If until range 10-100, (shown in the figure as 1E2) there are no or very less features, we want to exclude until that range and consider from range (100-1000), or, in other words, we take 'range:100-1000 or 1E3 or 1000' as Cutoff_LOD. This value will be used to replace the zeros in the data table
7. Once we consider that LOD value,
    - We create  a temparory dataset with all the feature intensites of your sample (not the ctrl, only the sample) and checking it against the cutoff_LOD value. If it is less than the cutoff_LOD, we replace it with cutoff_LOD. Thus, for instance, if we take 1000 as cutoff_LOD, our sample data will be filled with a bunch of 1000s now instead of zeros
    - Then we create a Final dataset using the temporary dataset. Here, we try to see if each feature from all samples is noise or not (from step 4), if it noise, we replace the feature with cutoff_LOD, else we keep the info from the temporary dataset.


For the code below, <font color='red'> I selected 'N' and gave a LOD value: 1000 </font>

In [ ]:
#Enter the LOD value as seen in the frequency plot: Ex: 1E3 or 1000
Cutoff_LOD <-ifelse(readline("Was Imputation step already performed? Y/N")=="Y",RawLOD,as.numeric(readline("Enter your Cutoff LOD here:")))  

# Replacing the Sample intensities with Cutoff_LOD, if they are lower than Cutoff_LOD
temp_matrix <- c()
for (i in 1:ncol(Samples)){ 
    x <- ifelse(Samples[,i] > Cutoff_LOD, Samples[,i],Cutoff_LOD)
    temp_matrix <- cbind(temp_matrix,as.matrix(x))
  }
colnames(temp_matrix) <- colnames(Samples)

# Replacing the Sample intensities with Cutoff_LOD, if they are considered as a noise(bg_bin==1)
Final_matrix <-c()
for (i in 1:ncol(temp_matrix)){
    x <-ifelse(input_data1[,ncol(input_data1)] ==1, Cutoff_LOD, temp_matrix[,i])
    Final_matrix <-cbind(Final_matrix,x)
  }
colnames(Final_matrix) <- colnames(Samples)


write.csv(Final_matrix,file=paste0('Processed_Quant_Table_filled_with_Value_',Cutoff_LOD ,'.csv'),row.names =FALSE)

## Normalization:
The following code performs sample-centric (column-wise) normalisation:

In [ ]:
#Sample centric normalisation:----------------------------------------------- I said "Y" (YES) here
if (readline("Do you want to perform Normalization: Y/N:") == 'Y'){  
  sample_sum <- colSums(Final_matrix, na.rm= TRUE, dims = 1)
  Normalized_data <- c()
  for (i in 1:ncol(Final_matrix)){
    x <- Final_matrix[,i] / sample_sum[i]
    Normalized_data <- cbind(Normalized_data, x)
  }
  
  colnames(Normalized_data) <- names(sample_sum)
  
  
} else return(Final_matrix)

print(paste('No.of NA values in Normalized data:',sum(is.na(Normalized_data)== TRUE)))
#write.csv(Normalized_data,file=paste0('CCE_Normalised_Data' ,'.csv'),row.names =FALSE)

## MDS Plots
The goal of Multi-Dimensional scaling (MDS) is to visualize multivariate data in 2D plots

In [ ]:
#excluding Blanks from md and some other filtering conditions
MetaData <-  md %>% filter(ATTRIBUTE_Filament_Possition != 'Blank',
                           ATTRIBUTE_Location!='Santa_Barbara_Basin',
                           ATTRIBUTE_Location!='Transcet_1',
                           ATTRIBUTE_Location!='Transcet_2',
                           ATTRIBUTE_Location!='Transect_3',
                           ATTRIBUTE_Depth <= 20 ) 

# the corresponding column files for the filtered MetaData is picked from the normalized data
md_data <- Normalized_data[,which(colnames(Normalized_data)%in%rownames(MetaData))] 

In [ ]:
head(MetaData)
dim(MetaData)

In [ ]:
head(md_data)
dim(md_data)

In [ ]:
#For PCA plots, use Euclidean distance:
#dist_matrix <- dist(t(md_data), method="euclidean")

#For PCoA plots using Bray-curtis distance:
dist_matrix <- bcdist(t(md_data)) # transposed in order to compute the distance between the columns of a data matrix

In [ ]:
pcoa<- cmdscale(dist_matrix, eig = TRUE, x.ret=TRUE)
pcoa.var.per <-round(pcoa$eig/sum(pcoa$eig)*100,1)
pcoa.values <- pcoa$points
pcoa.data <- data.frame(MetaData$ATTRIBUTE_Depth_Range,
                        X=pcoa.values[,1],
                        Y=pcoa.values[,2])

In [ ]:
##GGPLOT
Plot <- ggplot(pcoa.data, aes(x=X, y=Y, col= as.factor(MetaData$ATTRIBUTE_Filament_Possition))) + 
  #geom_jitter(aes(shape = as.factor(MetaData$ATTRIBUTE_Depth)), size = 3) +
  geom_point(size=2,alpha=0.8)  +
  ggtitle(label="MDS plot") +
  xlab(paste0("MDS1 : ",pcoa.var.per[1],"%",sep="")) + 
  ylab(paste0("MDS2 : ",pcoa.var.per[2],"%",sep="")) + 
  labs(color = "Cycles",shape='Depth Range') + 
  theme(plot.title = element_text(hjust = 0.5)) 


In [ ]:
Plot <- Plot + labs(subtitle = 'Using Bray_Curtis Distance on normalized data')
Plot
#ggsave(file="PCoA_Depth_0_20.svg", plot=Plot, width=10, height=8)

In [ ]:
#To visualize the color palette

#options(repr.plot.width=5, repr.plot.height=6) #'global' settings for plot size in the output cell
#display.brewer.all()

In the below code, n=5 is selected as default. Because, in general, the palette breaks the color from darker shade to lighter. In order to avoid considering really light colors in the plot (as it is hard to see), we specify a random n=5 as it is greater than no.of days (Max no.of days in a cycle=4) and take the first few colors according to the no.of days in each cycle

In [ ]:
a1 <- rev(brewer.pal(5, "YlOrRd"))
b1 <- rev(brewer.pal(5, "PuBu"))
c1 <- rev(brewer.pal(5, "Greens"))
d1 <- rev(brewer.pal(5, "RdPu"))

In [ ]:
plot(rep(1,length(b1)),col= b1,pch=19,cex=3,ylab=NULL,xlab=NULL) #For visualising the gradient colors in a plot

In [ ]:
a1 <- a1[1:length(grep(Primary_level[1],levels(col_used)))]
b1 <- b1[1:length(grep(Primary_level[2],levels(col_used)))]
c1 <- c1[1:length(grep(Primary_level[3],levels(col_used)))]
d1 <- d1[1:length(grep(Primary_level[4],levels(col_used)))]

colors_gradient <- c(a1,b1,c1,d1)
colors_gradient

In [ ]:
plot(rep(1,length(b1)),col= b1,pch=19,cex=3,ylab=NULL,xlab=NULL) #For visualising the gradient colors in a plot

In [ ]:
Plot2 <- ggplot(pcoa.data, aes(x=X, y=Y, col= as.factor(MetaData$ATTRIBUTE_Location))) + 
  geom_point(size=2,alpha=0.8)  +
  ggtitle(label="MDS plot") +
  xlab(paste0("MDS1 : ",pcoa.var.per[1],"%",sep="")) + 
  ylab(paste0("MDS2 : ",pcoa.var.per[2],"%",sep="")) + 
  labs(color = "Location") + 
  theme(plot.title = element_text(hjust = 0.5)) 

In [ ]:
Plot2 <- Plot2 + scale_color_manual(values = colors_gradient)+
  labs(subtitle = 'Using Bray_Curtis Distance on normalized data')

Plot2

In [ ]:
#To save a ggplot as svg (scaled vector graph)
ggsave(file="PCoA_Depth_0_20.svg", plot=Plot2, width=10, height=8)